# Predict future sales

## Import libraries

In [25]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import scipy.sparse 
import lightgbm as lgb
import math
import gc
from itertools import product
from tqdm import tqdm_notebook


for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.18.5
pandas 1.1.4
scipy 1.5.4
sklearn 0.19.0
lightgbm 3.1.0


In [2]:
def downcast_dtypes(df, inplace=False):
    if(inplace == False):
        df = df.copy()
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    date_cols =  [c for c in df if (c == 'date' and df[c].dtype == "object")]
        
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    for date_col in date_cols:
        df[date_col] = pd.to_datetime(df[date_col])
    
    if(inplace == False):
        return df
    
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

## Load source data

In [3]:
sales_train = pd.read_csv('../data/sales_train.csv.gz')
sales_test = pd.read_csv('../data/test.csv.gz')
items = pd.read_csv('../data/items.csv')
item_categories = pd.read_csv('../data/item_categories.csv')
shops = pd.read_csv('../data/shops.csv')

downcast_dtypes(sales_train, inplace=True)
downcast_dtypes(sales_test, inplace=True)
downcast_dtypes(items, inplace=True)
downcast_dtypes(item_categories, inplace=True)
downcast_dtypes(shops, inplace=True)

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = [] 
for block_num in sales_train['date_block_num'].unique():
    cur_shops = sales_train.loc[sales_train['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales_train.loc[sales_train['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)
downcast_dtypes(grid, inplace=True)
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [5]:
# Let's calculate summary sales of each item in each shop during a month
gb = sales_train.groupby(index_cols, as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day': 'target'})
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)
downcast_dtypes(all_data, inplace=True)

# Let's calculate summary sales of all items in each shop during a month
gb = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day':'target_shop'})
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)
downcast_dtypes(all_data, inplace=True)

# Let's calculate summary sales of each item in all shops during a month
gb = sales_train.groupby(['item_id', 'date_block_num'],as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day': 'target_item'})
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)
downcast_dtypes(all_data, inplace=True)

In [6]:
all_data.head(20)

,shop_id,item_id,date_block_num,target,target_shop,target_item
0,59,22154,0,1.0,2017.0,18.0
1,59,2552,0,0.0,2017.0,0.0
2,59,2554,0,0.0,2017.0,1.0
3,59,2555,0,0.0,2017.0,2.0
4,59,2564,0,0.0,2017.0,5.0
5,59,2565,0,0.0,2017.0,2.0
6,59,2572,0,0.0,2017.0,12.0
7,59,2573,0,0.0,2017.0,16.0
8,59,2574,0,2.0,2017.0,119.0
9,59,2593,0,0.0,2017.0,5.0


In [7]:
del grid, gb 
gc.collect();

In [8]:
# Get all fields used to add indicators of former times
# Actually these are all the fields of all_data apart from the ones from list index_cols
cols_to_rename = list(all_data.columns.difference(index_cols)) 
cols_to_rename

['target', 'target_item', 'target_shop']

In [9]:
shift_range = [1, 2, 3, 4, 5, 12]
for month_shift in tqdm_notebook(shift_range):    
    # Let's create a temporary table to calculate shifted data
    train_shift = all_data[index_cols + cols_to_rename].copy()
    downcast_dtypes(train_shift)
    # Fill the values of months to shift taking the part of the source table which corresponds to the shift we need
    # The trick here is to take the part of table which is indexed with current date_block_num, but contains
    # data for the shifted months (date_block_num + month_shift)
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    # Create a function to rename shifted columns to make them clear the shift value
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)    
    downcast_dtypes(train_shift)

    # Merge temporary table and source table
    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
    downcast_dtypes(all_data)
    gc.collect()

del train_shift
gc.collect()

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


15

In [11]:
# Сохраним список добавленных полей со сдвигами, чтобы потом удалить их при обучении модели
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Удалим ненужные категориальные признаки
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()
downcast_dtypes(item_category_mapping, inplace=True)

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
downcast_dtypes(all_data, inplace=True)
gc.collect();

In [12]:
convert_size(all_data.memory_usage().sum())

'1.1 GB'

In [13]:
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [14]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

In [26]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_test.values)

print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))

Test R-squared for linreg is 0.260629


In [27]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model.predict(X_test)

print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.144712 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Test R-squared for LightGBM is 0.303581
